In [182]:
import pandas as pd
import seaborn as sns
import numpy as np
from collections import Counter

# import sys
# if sys.version_info >= (3, 7):
    # sys.stdout.reconfigure(line_buffering=False)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
np.set_printoptions(threshold=np.inf)


In [124]:
# Импорт набора данных для проверки качества работы алгоритмов классификации
from sklearn.datasets import load_iris

In [125]:
def round_to_3(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.

    Аргументы:
        x: Число.

    Возвращаемое значение:
        Результат округления числа до 3 знаков после запятой.
    """

    return round(x, 3)

In [126]:
def custom_compare(x, y):
    if str(x) != str(y):
        raise RuntimeError(f'Ожидаемое значение: {y}. Фактическое: {x}')

# Дерево принятяи решений

In [127]:
class Node:
    def __init__(self,
                 factor_ind,
                 threshold,
                 left_node,
                 right_node):
        """
        Конструктор объекта «вершина дерева».

        Аргументы:
            factor_ind: Индекс фактора, про который задаётся вопрос в вершине.
            threshold: Пороговое значение, с которым сравнивается значение фактора в вершине.
            left_node: Вершина, содержащая левое поддерево текущей вершины.
            right_node: Вершина, содержащая правое поддерево текущей вершины.
        """

        self.factor_ind = factor_ind
        self.threshold = threshold

        self.left_node = left_node
        self.right_node = right_node

    def is_leaf(self):
        """
        Возвращает значение False, которое соответствует тому, что объект не является листом.
        """

        return False

class Leaf:
    def __init__(self,
                 predicted_value):
        """
        Конструктор объекта «лист дерева».

        Аргументы:
            predicted_value: Значение, которое является предсказанием в данной листе.
        """

        self.predicted_value = predicted_value

    def is_leaf(self):
        """
        Возвращает значение True, которое соответствует тому, что объект является листом.
        """

        return True

In [128]:
def print_tree(cur_node, s=''):
    """
    Производит печать на экран дерева, представленного в виде вершин и листьев.

    Аргументы:
        cur_node: Корневая вершина дерева, которое нужно напечатать.
        s: Опциональный параметр. Задаёт отступ при печати

    Возвращаемое значение:
        На экран будет выведено дерево в следующем формате:

        Вершина 0 (корневая вершина)
            Вершина 1 (левая вершина от вершины 0)
                ... и так далее до листьев
            Вершина 2 (правая вершина от вершины 0)
                ... и так далее до листьев
    """

    if cur_node.is_leaf():
        print(s + f'Лист(предсказание={cur_node.predicted_value})')
        return

    print(s + f'Вершина(фактор={cur_node.factor_ind}, порог={cur_node.threshold})')

    print_tree(cur_node.left_node, s + '    ')
    print_tree(cur_node.right_node, s + '    ')

In [129]:
def decision_tree_predict_solution(cur_node, x):
    """
    Производит предсказание значения для объекта x с помощью заданного дерева принятия решений.

    Аргументы:
        cur_node: Корневая вершина дерева, с помощью которого производится предсказание.
        x: Объект, для которого происходит предсказание.

    Возвращаемое значение:
        Предсказанное значение для объекта x.
        Если в листьях дерева записан класс, то возвращается класс,
        если в листьях дерева записано численное значение, то возвращается число.
    """
    if cur_node.is_leaf():
        return cur_node.predicted_value
    if x[cur_node.factor_ind] >= cur_node.threshold:
        return decision_tree_predict_solution(cur_node.right_node, x)
    return decision_tree_predict_solution(cur_node.left_node, x)


decision_tree_predict_test()

Тест прошёл успешно!


In [130]:
def decision_tree_predict_test():
    tree_example_1 = \
        Node(0, 10,
             Node(1, -1,
                  Leaf(1),
                  Leaf(2)),
             Node(0, 15,
                  Leaf(2),
                  Leaf(3)))
    x_example_1 = np.array([12, 2])
    res_example_1 = 2

    custom_compare(decision_tree_predict_solution(tree_example_1, x_example_1), res_example_1)

    tree_example_2 = \
        Node(2, -1,
             Node(3, 7,
                  Node(1, 2,
                       Leaf(1.1),
                       Node(3, 0,
                            Leaf(-1.3),
                            Leaf(1.7))),
                  Node(2, -6,
                       Leaf(2.2),
                       Leaf(-7.3))),
             Node(0, 15,
                  Leaf(2),
                  Leaf(3)))
    x_example_2 = np.array([2, 3, -4, 0])
    res_example_2 = 1.7

    custom_compare(decision_tree_predict_solution(tree_example_2, x_example_2), res_example_2)

    print('Тест прошёл успешно!')

In [131]:
decision_tree_predict_test()

Тест прошёл успешно!


In [132]:
def gini_index_solution(X):
    """
    Производит вычисление хаотичности данных с помощью критерия Джини.

    Аргументы:
        X: Набор объектов, представленный в виде матрицы размера n x m.
           Каждая строка матрицы соответствует объекту.
           Каждый объект описывается (m - 1) фактором, а также классом, которому принадлежит.
           Класс представляется числом.
           Индекс класса в строке объекта — (m - 1), если считать, что индексация в векторе идёт от 0.

    Возвращаемое значение:
        Вычисленное значение хаотичности данных, округлённое до 3 знаков после запятой.
    """
    if not len(X):
        return 0
    gini_sum = 0
    classes = list(set(X[:, -1]))
    for c in classes:
        pk = len(X[X[:, -1] == c]) / len(X)
        gini_sum += pk * (1 - pk)
    return round_to_3(gini_sum)



gini_index_test()

Тест прошёл успешно!


In [133]:
def gini_index_test():
    X_example_1 = np.array([
        [1, 2, -1],
        [2, 3, 1],
        [5, 4, 1],
        [2, 5, 1]
    ])
    res_example_1 = 0.375

    custom_compare(gini_index_solution(X_example_1), res_example_1)

    X_example_2 = np.array([
        [1, 1, 2, 3],
        [2, 2, 3, 1],
        [5, 5, 4, 2],
        [2, 2, 5, 1],
        [1, 1, 2, 3],
        [2, 2, 3, 2],
        [5, 5, 4, 2],
        [2, 2, 5, 1],
        [1, 1, 2, 3],
        [2, 2, 3, 1],
        [5, 5, 4, 1],
        [2, 2, 5, 1]
    ])
    res_example_2 = 0.625

    custom_compare(gini_index_solution(X_example_2), res_example_2)

    print('Тест прошёл успешно!')

In [134]:
gini_index_test()

Тест прошёл успешно!


In [135]:
def data_split_solution(X, factor_ind, factor_value):
    """
    Производит разбиение данных на две части в зависимости от значения конкретного фактора
    по следующему правилу: объекты в X, для которых значение фактора с индексом factor_ind больше или равно factor_value,
    идут в правую часть разбиения, остальные — в левую.

    Аргументы:
        X: Набор объектов, представленный в виде матрицы размера n x m.
           Каждая строка матрицы соответствует объекту.
           Каждый объект описывается (m - 1) фактором, а также классом, которому принадлежит.
           Класс представляется числом.
           Индекс класса в строке объекта — (m - 1), если считать, что индексация в векторе идёт от 0.
        factor_ind: Индекс фактора, по которому происходит разбиение.
        factor_value: Пороговое значение, по которому происходит разбиение.

    Возвращаемое значение:
        Пара из X_l и X_r, где X_l и X_r — левая и правая части разбиения, соответственно.
        Важно, что порядок объектов в X_l и X_r должен быть таким же, как в X.
        То есть если какие-то две строчки шли друг за другом в X, а после разбиения вместе оказались,
        например, в X_l, то идти друг за другом в X_l они должны в том же порядке, в котором шли в X.
    """
    left_res = []
    right_res = []
    for x in X:
        if x[factor_ind] >= factor_value:
            right_res.append(x)
        else:
            left_res.append(x)
    return (np.array(left_res), np.array(right_res))


data_split_test()

Тест прошёл успешно!


In [136]:
def data_split_test():
    X_example_1 = np.array([
        [1, 2, -1],
        [2, 3, 1],
        [5, 4, 1],
        [2, 5, 1]
    ])
    factor_ind_example_1 = 0
    factor_value_example_1 = 2

    X_l_example_1 = np.array([
        [1, 2, -1]
    ])
    X_r_example_1 = np.array([
        [2, 3, 1],
        [5, 4, 1],
        [2, 5, 1]
    ])

    res_example_1 = (X_l_example_1, X_r_example_1)

    custom_compare(data_split_solution(X_example_1, factor_ind_example_1, factor_value_example_1),
                   res_example_1)

    X_example_2 = np.array([
        [1, 1, 2, 3],
        [2, 2, 3, 1],
        [5, 5, 4, 2],
        [2, 2, 5, 1],
        [1, 1, 2, 3],
        [2, 2, 3, 2],
        [5, 5, 4, 2],
        [2, 2, 5, 1],
        [1, 1, 2, 3],
        [2, 2, 3, 1],
        [5, 5, 4, 1],
        [2, 2, 5, 1]
    ])
    factor_ind_example_2 = 2
    factor_value_example_2 = 4

    X_l_example_2 = np.array([
        [1, 1, 2, 3],
        [2, 2, 3, 1],
        [1, 1, 2, 3],
        [2, 2, 3, 2],
        [1, 1, 2, 3],
        [2, 2, 3, 1]
    ])
    X_r_example_2 = np.array([
        [5, 5, 4, 2],
        [2, 2, 5, 1],
        [5, 5, 4, 2],
        [2, 2, 5, 1],
        [5, 5, 4, 1],
        [2, 2, 5, 1]
    ])

    res_example_2 = (X_l_example_2, X_r_example_2)

    custom_compare(data_split_solution(X_example_2, factor_ind_example_2, factor_value_example_2),
                   res_example_2)

    print('Тест прошёл успешно!')

In [137]:
data_split_test()

Тест прошёл успешно!


In [138]:
def optimal_split_parameters_grid_search_solution(X):
    """
    Находит оптимальные параметры разбиения данных на левую и правую части.
    Оптимальность разбиения определяется с помощью критерия Джини.

    Аргументы:
        X: Набор объектов, представленный в виде матрицы размера n x m.
           Каждая строка матрицы соответствует объекту.
           Каждый объект описывается (m - 1) фактором, а также классом, которому принадлежит.
           Класс представляется числом.
           Индекс класса в строке объекта — (m - 1), если считать, что индексация в векторе идёт от 0.

    Возвращаемое значение:
        Пара из индекса фактора и порогового значения,
        при которых достигается оптимальное разбиение данных.
    """
    origin_gini = gini_index_solution(X)
    # print(f"gini:{origin_gini}")
    best_score = float('-inf')
    best_idx = -1
    best_factor = -1
    for factor_idx in range(len(X[0]) - 1):
        factor_values = list(set(X[:, factor_idx]))
        for val in factor_values:
            left_res, right_res = data_split_solution(X, factor_idx, val)
            if not len(left_res) or not len(right_res):
                continue
            left_gini = gini_index_solution(left_res)
            right_gini = gini_index_solution(right_res)
            score = origin_gini - len(left_res) * left_gini / len(X) - len(right_res) * right_gini / len(X)
            # print(f"left gini:{left_gini}, left_len:{len(left_res)}, right_len:{len(right_res)} right_gini:{right_gini    }")
            if score > best_score:
                best_score = score
                best_idx = factor_idx
                best_factor = val
                # print(f"score {best_score}, idx:{best_idx}, factor:{best_factor}")
    return (best_idx, best_factor)


optimal_split_parameters_grid_search_test()

Тест прошёл успешно!


In [139]:
def optimal_split_parameters_grid_search_test():
    X_example_1 = np.array([
        [1, 2, -1],
        [2, 3, 1],
        [5, 4, 1],
        [2, 5, 1]
    ])
    X_example_1 = X_example_1.astype('float')

    factor_ind_example_1 = 0
    factor_value_example_1 = 2.0

    res_example_1 = (factor_ind_example_1, factor_value_example_1)

    custom_compare(optimal_split_parameters_grid_search_solution(X_example_1),
                   res_example_1)

    X_example_2 = np.array([
        [-7,  2,  8,  1],
        [ 6,  3, -3,  2],
        [ 3,  0,  4,  3],
        [ 8,  3,  0,  1],
        [-9,  4, -8,  2],
        [-4,  -2, -9,  3],
        [-2,  7,  6,  1],
        [-6,  -2,  5,  2],
        [-2,  -1,  3,  3],
        [-3,  3, -6,  1]
    ])
    X_example_2 = X_example_2.astype('float')

    factor_ind_example_2 = 1
    factor_value_example_2 = 2.0

    res_example_2 = (factor_ind_example_2, factor_value_example_2)

    custom_compare(optimal_split_parameters_grid_search_solution(X_example_2),
                   res_example_2)

    X_example_3 = np.array([
        [4, 3, 1],
        [3, 4, 1],
        [2, 2, 0]
    ])
    X_example_3 = X_example_3.astype('float')

    factor_ind_example_3 = 0
    factor_value_example_3 = 3.0

    res_example_3 = (factor_ind_example_3, factor_value_example_3)

    custom_compare(optimal_split_parameters_grid_search_solution(X_example_3),
                   res_example_3)

    print('Тест прошёл успешно!')

In [140]:
optimal_split_parameters_grid_search_test()

Тест прошёл успешно!


In [189]:
second_passed = False


def build_node_solution(X, cur_depth=1, max_depth=None):
    """
    Строит дерево принятия решений.

    Аргументы:
        X: Набор объектов, представленный в виде матрицы размера n x m.
           Каждая строка матрицы соответствует объекту.
           Каждый объект описывается (m - 1) фактором, а также классом, которому принадлежит.
           Класс представляется числом.
           Индекс класса в строке объекта — (m - 1), если считать, что индексация в векторе идёт от 0.
        cur_depth: Опциональный параметр, по умолчанию равен 1.
                   Глубина, на которой строится текущая вершина.
        max_depth: Опциональный параметр, по умолчанию равен None.
                   Максимальная глубина дерева.
                   Если значение параметра равно None, то на глубину дерева
                   не накладывается ограничений в процессе построения дерева.

    Возвращаемое значение:
        Построенное на данных из X дерево принятия решений, которое либо состоит
        из вершины, к которой присоединены рекурсивно построенные поддеревья,
        либо состоит из одного листа с предсказанием.
    """
    # print(bool(max_depth))
    if max_depth:
        if cur_depth > max_depth:
            class_counts = Counter(X[:, -1])  # Подсчитываем количество каждого класса
            most_common_class = class_counts.most_common(1)[0][0]  # Выбираем наиболее часто встречающийся класс
            # if second_passed:
            print(f"max depth leaf:{most_common_class}")
            return Leaf(most_common_class)
    else:
        if all(x[-1] == X[0][-1] for x in X):
            if second_passed:
                print("solo class leaf", X[0][-1])
            # if X[0][-1] == 3:
                # print("DFVSFSFSFDFVSFSFSFDFVSFSFSFDFVSFSFSFDFVSFSFSFDFVSFSFSFDFVSFSFSF")
            return Leaf(X[0][-1])

    # if cur_depth - 1 == max_depth:
        # print("max depth leaf class", min(set(X[:][-1])), "with data", X)
        # return Leaf(min(set(X[:][-1])))

    factor_idx, threshold = optimal_split_parameters_grid_search_solution(X)
    left_X, right_X = data_split_solution(X, factor_idx, threshold)
    left_node = build_node_solution(left_X, cur_depth + 1, max_depth)
    right_node = build_node_solution(right_X, cur_depth + 1, max_depth)
    node = Node(factor_idx, threshold, left_node, right_node)
    # print("node", node.factor_ind, node.threshold)
    return node



# res = build_node_solution(X_example_1, max_depth=max_depth_example_1)
build_node_test()

max depth leaf:1.0
max depth leaf:0.0
max depth leaf:1.0
max depth leaf:2.0
values 1 1 euqal is True
values 0 0 euqal is True
values 1 1 euqal is True
values 2 2 euqal is True
-------------first test sucessfully passed-------------
values 0 0 euqal is True
values 1 1 euqal is True
values 2 2 euqal is True
values 2 2 euqal is True
values 1 1 euqal is True
values 2 2 euqal is True
values 1 1 euqal is True
values 2 2 euqal is True
values 2 2 euqal is True
-------------second test sucessfully passed-------------
max depth leaf:2
max depth leaf:3
values 2 2 euqal is True
values 3 2 euqal is False


AssertionError: 

In [187]:
def compare_trees(tree_1, tree_2):
    """
    Проверяет два дерева на равенство.

    Аргументы:
        tree_1: Первое дерево.
        tree_2: Второе дерево.

    Возвращаемое значение:
        True, если деревья равны.
        False в противном случае.
    """

    if tree_1.is_leaf() != tree_2.is_leaf():
        print(f"one is leaf, but second is node")
        return False

    if tree_1.is_leaf() and tree_2.is_leaf():
        print(f"values {int(tree_1.predicted_value)} {int(tree_2.predicted_value)} euqal is {int(tree_1.predicted_value) == int(tree_2.predicted_value)}")
        return int(tree_1.predicted_value) == int(tree_2.predicted_value)

    return tree_1.factor_ind == tree_2.factor_ind and \
        tree_1.threshold == tree_2.threshold and \
        compare_trees(tree_1.left_node, tree_2.left_node) and \
        compare_trees(tree_1.right_node, tree_2.right_node)


def build_node_test():
    iris = load_iris()

    X = iris.data
    y = iris.target.reshape((len(X), 1))

    X_example_1 = np.hstack([X[:, :2], y])
    max_depth_example_1 = 2

    res_example_1 = \
        Node(0, 5.5,
             Node(1, 2.9,
                 Leaf(1.0),
                 Leaf(0.0)),
             Node(0, 6.3,
                 Leaf(1.0),
                 Leaf(2.0)))

    assert compare_trees(build_node_solution(X_example_1, max_depth=max_depth_example_1), res_example_1)
    print("-------------first test sucessfully passed-------------")

    X_example_2 = np.hstack([X, y])
    max_depth_example_2 = None

    res_example_2 = \
        Node(2, 3.0,
             Leaf(0.0),
             Node(3, 1.8,
                 Node(2, 5.0,
                     Node(3, 1.7,
                         Leaf(1.0),
                         Leaf(2.0)),
                     Node(3, 1.6,
                         Leaf(2.0),
                         Node(0, 7.2,
                             Leaf(1.0),
                             Leaf(2.0)))),
                 Node(2, 4.9,
                     Node(0, 6.0,
                         Leaf(1.0),
                         Leaf(2.0)),
                     Leaf(2.0))))

    assert compare_trees(build_node_solution(X_example_2, max_depth=max_depth_example_2), res_example_2)
    print("-------------second test sucessfully passed-------------")
    second_passed = True


    X_example_3 = np.array([
        [-1, 2],
        [1, 3],
        [2, 2],
        [3, 3],
        [4, 2]
    ])
    max_depth_example_3 = 1

    res_example_3 = \
        Node(0, 1.0,
             Leaf(2.0),
             Leaf(2.0))

    assert compare_trees(build_node_solution(X_example_3, max_depth=max_depth_example_3), res_example_3)

    print('Тест прошёл успешно!')

In [143]:
build_node_test()

max depth leaf 2.0
max depth leaf 1.0
node 1 2.9
max depth leaf 2.0
max depth leaf 2.0
node 0 6.3
node 0 5.5


AssertionError: 